In [ ]:
import geopandas as gpd

In [ ]:
import folium

# Filtrar los datos para los distritos San Miguel y Pueblo Libre
filtered_data = quadkeys_final_final[quadkeys_final_final['DISTRITO'].isin(['MAGDALENA DEL MAR', 'SAN MIGUEL'])]

# Asegurarnos de que el CRS es EPSG:4326 para Folium
if filtered_data.crs.to_string() != 'EPSG:4326':
    filtered_data = filtered_data.to_crs(epsg=4326)

# Crear el mapa centrado en Lima
mapa = folium.Map(location=[-12.079, -77.058], zoom_start=14, tiles="cartodbpositron")

# Función para asignar colores según el distrito
def color_por_distrito(distrito):
    if distrito == 'MAGDALENA DEL MAR':
        return 'blue'
    elif distrito == 'SAN MIGUEL':
        return 'green'
    return 'gray'

# Añadir las grillas al mapa
folium.GeoJson(
    data=filtered_data.to_json(),
    name="Grillas de Velocidad",
    style_function=lambda feature: {
        'fillColor': color_por_distrito(feature['properties']['DISTRITO']),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['quadkey', 'avg_d_kbps', 'DISTRITO'],
        aliases=['Quadkey:', 'Vel. Descarga (kbps):', 'Distrito:']
    )
).add_to(mapa)

# Añadir control de capas
folium.LayerControl().add_to(mapa)

mapa

In [ ]:
import folium
import branca.colormap as cm

# 1. Filtrar datos para un periodo específico
periodo_especifico = quadkeys_final_final[(quadkeys_final['year'] == 2019) & (quadkeys_final['quarter'] == 3)]

# 2. Asegurarte de que el CRS sea EPSG:4326
if periodo_especifico.crs is None or periodo_especifico.crs.to_string() != 'EPSG:4326':
    periodo_especifico = periodo_especifico.to_crs(epsg=4326)

# 3. Verificar geometrías válidas
periodo_especifico = periodo_especifico[periodo_especifico.geometry.notnull()]  # Quitar geometrías nulas
periodo_especifico = periodo_especifico[periodo_especifico.is_valid]  # Quitar geometrías inválidas

# 4. Crear el mapa con Folium
m = folium.Map(location=[-12.0464, -77.0428], zoom_start=10)

# 5. Normalizar los colores según avg_d_kbps
min_speed = periodo_especifico['avg_d_kbps'].min()
max_speed = periodo_especifico['avg_d_kbps'].max()
colormap = cm.linear.YlOrRd_09.scale(min_speed, max_speed)

# 6. Convertir a GeoJSON asegurando que las propiedades están incluidas
# Las columnas que se desean en el tooltip deben estar presentes en el GeoDataFrame
geojson_data = periodo_especifico[['quadkey', 'avg_d_kbps', 'geometry']].to_json()

# 7. Agregar las geometrías al mapa
folium.GeoJson(
    geojson_data,
    name="Velocidad de descarga",
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['avg_d_kbps']),
        'color': 'black',
        'weight': 0.2,
        'fillOpacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["quadkey","avg_d_kbps"],
        aliases=["Quadkey","Vel. Descarga (kbps)"],
        localize=True
    )
).add_to(m)

# 8. Agregar la barra de colores al mapa
colormap.caption = "Velocidad de Descarga (kbps)"
colormap.add_to(m)

# 9. Mostrar el mapa
m

In [ ]:
import folium

# Calcular el centro del mapa basado en los distritos de Lima y Callao
centro = [lima_callao_gdf.geometry.centroid.y.mean(), lima_callao_gdf.geometry.centroid.x.mean()]

# Crear el mapa base
mapa = folium.Map(location=centro, zoom_start=11, tiles="cartodbpositron")

# Agregar los límites distritales como capa GeoJSON
folium.GeoJson(
    lima_callao_gdf,
    name="Distritos de Lima y Callao",
    style_function=lambda x: {
        'color': 'blue',           # Color del borde
        'weight': 1.5,             # Grosor del borde
        'fillOpacity': 0.2         # Opacidad del relleno
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['DISTRITO', 'PROVINCIA'],  # Campos a mostrar en el tooltip
        aliases=['Distrito:', 'Provincia:']
    )
).add_to(mapa)

# Guardar el mapa interactivo como HTML
mapa.save("mapa_lima_callao.html")
print("Mapa interactivo guardado como 'mapa_lima_callao.html'.")
mapa


In [ ]:
filtered_data_2019_q3 = quadkeys_final_final1[(quadkeys_final_final1['year'] == 2019) & (quadkeys_final_final1['quarter'] == 4)]

In [ ]:
# Asegurarte de que el CRS sea EPSG:4326 (GeoJSON requiere este CRS)
if quadkeys_final_final.crs.to_string() != 'EPSG:4326':
    quadkeys_final_final = quadkeys_final_final.to_crs(epsg=4326)

# Exportar a GeoJSON
output_geojson_path = "quadkeys_final_final.geojson"
quadkeys_final_final.to_file(output_geojson_path, driver="GeoJSON")

print(f"Archivo GeoJSON exportado en: {output_geojson_path}")

In [ ]:
# Filtrar las grillas con valores faltantes en 'DISTRITO'
missing_distritos = quadkeys_final_final[quadkeys_final_final['DISTRITO'].isna()]

# Reproyectar a un CRS proyectado (EPSG:32718)
if missing_distritos.crs.to_string() != 'EPSG:32718':
    missing_distritos = missing_distritos.to_crs(epsg=32718)

if distrital_gdf.crs.to_string() != 'EPSG:32718':
    distrital_gdf = distrital_gdf.to_crs(epsg=32718)

# Calcular intersecciones
intersections = gpd.overlay(missing_distritos, distrital_gdf, how='intersection')

# Verificar las columnas disponibles en intersections
print("Columnas disponibles en intersections:", intersections.columns)

# Añadir el área de las intersecciones
intersections['intersect_area'] = intersections.geometry.area

# Encontrar el distrito con la mayor área de intersección para cada quadkey
max_intersections = (
    intersections.loc[intersections.groupby('quadkey')['intersect_area'].idxmax()]
)

# Verificar si 'DISTRITO' está presente en max_intersections
if 'DISTRITO' in max_intersections.columns:
    # Asignar el distrito correspondiente
    quadkeys_final_final = quadkeys_final_final.merge(
        max_intersections[['quadkey', 'DISTRITO']],
        on='quadkey',
        how='left',
        suffixes=('', '_imputed')
    )

    # Rellenar los valores faltantes en 'DISTRITO' con los imputados
    quadkeys_final_final['DISTRITO'] = quadkeys_final_final['DISTRITO'].fillna(
        quadkeys_final_final['DISTRITO_imputed']
    )

    # Eliminar la columna temporal 'DISTRITO_imputed'
    quadkeys_final_final = quadkeys_final_final.drop(columns=['DISTRITO_imputed'])
else:
    print("Error: La columna 'DISTRITO' no está disponible en las intersecciones.")

# Verificar el resultado final
print("Imputación completada. Número de valores faltantes restantes en 'DISTRITO':",
      quadkeys_final_final['DISTRITO'].isna().sum())


In [ ]:
quadkeys_final_final['avg_d_mbps'] = quadkeys_final_final['avg_d_kbps'] / 1000

In [ ]:
import folium

# Calcular centro
centro = [lima_callao_gdf.geometry.centroid.y.mean(), lima_callao_gdf.geometry.centroid.x.mean()]

mapa = folium.Map(location=centro, zoom_start=11, tiles="cartodbpositron")

# Primero agregar las grillas con los datos filtrados
folium.GeoJson(
    filtered_data_2019_q3,
    name="Grillas de Velocidad",
    style_function=lambda x: {
        'fillColor': 'green',
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.3  # Un poco más transparente
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['avg_d_mbps', 'PROVINCIA'],
        aliases=['Velocidad (Mbps):', 'Provincia:']
    )
).add_to(mapa)

# Ahora agregar los límites distritales con un estilo que resalte los bordes
def estilo_distritos(feature):
    distrito = feature['properties']['DISTRITO']
    if distrito in ["SAN MIGUEL", "PUEBLO LIBRE"]:
        return {'fillColor': 'none', 'color': 'red', 'weight': 2, 'fillOpacity': 0.0}
    else:
        return {'fillColor': 'none', 'color': 'blue', 'weight': 1.5, 'fillOpacity': 0.0}

folium.GeoJson(
    lima_callao_gdf,
    name="Distritos de Lima y Callao",
    style_function=estilo_distritos,
    tooltip=folium.GeoJsonTooltip(
        fields=['DISTRITO', 'PROVINCIA'],
        aliases=['Distrito:', 'Provincia:']
    )
).add_to(mapa)

folium.LayerControl().add_to(mapa)

mapa.save("mapa_lima_callao_con_grillas.html")
print("Mapa guardado con distritos encima y grillas semi-transparentes.")


In [ ]:
import folium
import branca
import geopandas as gpd

# Suponiendo que filtered_data_2019_q3 es tu GeoDataFrame filtrado
# con las columnas: 'avg_d_mbps'
# Asegúrate de tener una columna con la velocidad media (avg_d_mbps).

# Obtenemos los valores mínimo y máximo de la velocidad para escalar el colormap
min_speed = filtered_data_2019_q3['avg_d_mbps'].min()
max_speed = filtered_data_2019_q3['avg_d_mbps'].max()

# Creamos un colormap continuo (puedes cambiar la paleta a tu gusto, por ejemplo 'YlOrRd')
colormap = branca.colormap.linear.YlOrRd_09.scale(min_speed, max_speed)
colormap.caption = 'Velocidad de internet (Mbps)'

# Creamos el mapa
centro = [lima_callao_gdf.geometry.centroid.y.mean(), lima_callao_gdf.geometry.centroid.x.mean()]
mapa = folium.Map(location=centro, zoom_start=11, tiles="cartodbpositron")

# Capa de grillas con estilo dependiente de la velocidad
folium.GeoJson(
    filtered_data_2019_q3,
    name="Grillas de Velocidad",
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['avg_d_mbps']),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['avg_d_mbps', 'PROVINCIA'],
        aliases=['Velocidad (Mbps):', 'Provincia:']
    )
).add_to(mapa)

# Función de estilo para distritos (resaltarlos sin relleno)
def estilo_distritos(feature):
    distrito = feature['properties']['DISTRITO']
    if distrito in ["SAN MIGUEL", "PUEBLO LIBRE"]:
        return {'fillColor': 'none', 'color': 'red', 'weight': 2, 'fillOpacity': 0.0}
    else:
        return {'fillColor': 'none', 'color': 'blue', 'weight': 1.5, 'fillOpacity': 0.0}

# Capa de límites distritales
folium.GeoJson(
    lima_callao_gdf,
    name="Distritos de Lima y Callao",
    style_function=estilo_distritos,
    tooltip=folium.GeoJsonTooltip(
        fields=['DISTRITO', 'PROVINCIA'],
        aliases=['Distrito:', 'Provincia:']
    )
).add_to(mapa)

# Agregar colormap como leyenda
colormap.add_to(mapa)

folium.LayerControl().add_to(mapa)
mapa.save("mapa_lima_callao_con_grillas_colormap.html")
print("Mapa con leyenda guardado como 'mapa_lima_callao_con_grillas_colormap.html'.")

In [ ]:
# Reproyectar a CRS proyectado antes de calcular centroides
lima_callao_gdf = lima_callao_gdf.to_crs(epsg=32718)
centro = [lima_callao_gdf.geometry.centroid.y.mean(), lima_callao_gdf.geometry.centroid.x.mean()]
# Luego, si lo deseas, reproyecta de nuevo a EPSG:4326 para el mapa
lima_callao_gdf = lima_callao_gdf.to_crs(epsg=4326)


In [ ]:
colormap_html_str = colormap._repr_html_()

In [ ]:
import folium
import branca
import geopandas as gpd
from folium import Element
from pyproj import Transformer

# Suponiendo que ya tienes lima_callao_gdf y filtered_data_2019_q3 cargados en EPSG:4326

# 1. Reproyectamos a un CRS proyectado (por ejemplo EPSG:32718) para calcular centroides correctamente
lima_callao_proj = lima_callao_gdf.to_crs(epsg=32718)

# Calculamos centroides en el CRS proyectado
centroides = lima_callao_proj.geometry.centroid
centro_y = centroides.y.mean()
centro_x = centroides.x.mean()

# Convertimos el punto central desde EPSG:32718 a EPSG:4326 para Folium (lat, lon)
transformer = Transformer.from_crs("EPSG:32718", "EPSG:4326", always_xy=True)
centro_lon, centro_lat = transformer.transform(centro_x, centro_y)
centro = [centro_lat, centro_lon]

# Asegúrate que filtered_data_2019_q3 esté en EPSG:4326
# filtered_data_2019_q3 = filtered_data_2019_q3.to_crs(epsg=4326)

# Si existiera una columna 'centroid' u otra con geometrías, la removemos
if 'centroid' in filtered_data_2019_q3.columns:
    filtered_data_2019_q3 = filtered_data_2019_q3.drop(columns='centroid')

# 2. Creamos el colormap
min_speed = filtered_data_2019_q3['avg_d_mbps'].min()
max_speed = filtered_data_2019_q3['avg_d_mbps'].max()

colormap = branca.colormap.linear.YlOrRd_09.scale(min_speed, max_speed)
colormap.caption = 'Velocidad de internet (Mbps)'
colormap.orientation = 'vertical'  # Leyenda vertical

# Obtenemos el HTML del colormap usando _repr_html_()
colormap_html_str = colormap._repr_html_()

# 3. Creamos el mapa centrado
mapa = folium.Map(location=centro, zoom_start=11, tiles="cartodbpositron")

# 4. Añadimos las grillas usando .to_json() para evitar errores de serialización
folium.GeoJson(
    data=filtered_data_2019_q3.to_json(),
    name="Grillas de Velocidad",
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['avg_d_mbps']),
        'color': 'black',
        'weight': 0.5,
        'fillOpacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['avg_d_mbps', 'PROVINCIA'],
        aliases=['Velocidad (Mbps):', 'Provincia:']
    )
).add_to(mapa)

# Estilo para distritos
def estilo_distritos(feature):
    distrito = feature['properties']['DISTRITO']
    if distrito in ["SAN MIGUEL", "PUEBLO LIBRE"]:
        return {'fillColor': 'none', 'color': 'red', 'weight': 2, 'fillOpacity': 0.0}
    else:
        return {'fillColor': 'none', 'color': 'blue', 'weight': 1.5, 'fillOpacity': 0.0}

# Añadimos límites distritales
folium.GeoJson(
    data=lima_callao_gdf.to_json(),
    name="Distritos de Lima y Callao",
    style_function=estilo_distritos,
    tooltip=folium.GeoJsonTooltip(
        fields=['DISTRITO', 'PROVINCIA'],
        aliases=['Distrito:', 'Provincia:']
    )
).add_to(mapa)

# 5. Agregamos estilo para la leyenda vertical a la derecha
style = """
<style>
.mapLegend {
  position: absolute;
  top: 10px;        /* Ajusta verticalmente */
  right: 10px;       /* Ajusta horizontalmente */
  background-color: rgba(255, 255, 255, 0.8);
  padding: 10px;
  border-radius: 5px;
  box-shadow: 0 0 5px rgba(0,0,0,0.3);
  z-index: 9999;
  text-align: center;
}
.caption {
  font-weight: bold;
  margin-top: 5px;
}
</style>
"""

mapa.get_root().html.add_child(Element(style))

legend_html = f"""
<div class='mapLegend'>
    {colormap_html_str}
    <div class='caption'>{colormap.caption}</div>
</div>
"""

mapa.get_root().html.add_child(Element(legend_html))

# Control de capas
folium.LayerControl().add_to(mapa)

# Guardamos el mapa
mapa.save("mapa_lima_callao_con_grillas_colormap.html")
print("Mapa con correcciones guardado como 'mapa_lima_callao_con_grillas_colormap.html'.")


In [ ]:
import folium
import branca
import geopandas as gpd
from folium import Element
from pyproj import Transformer

# Suponiendo que lima_callao_gdf y filtered_data_2019_q3 están cargados en EPSG:4326

# 1. Reproyectar para calcular el centro correctamente (opcional si no es esencial)
lima_callao_proj = lima_callao_gdf.to_crs(epsg=32718)
centroides = lima_callao_proj.geometry.centroid
centro_y = centroides.y.mean()
centro_x = centroides.x.mean()

transformer = Transformer.from_crs("EPSG:32718", "EPSG:4326", always_xy=True)
centro_lon, centro_lat = transformer.transform(centro_x, centro_y)
centro = [centro_lat, centro_lon]

# Si 'centroid' existe en filtered_data_2019_q3 la removemos
if 'centroid' in filtered_data_2019_q3.columns:
    filtered_data_2019_q3 = filtered_data_2019_q3.drop(columns='centroid')

# 2. Crear el colormap
min_speed = filtered_data_2019_q3['avg_d_mbps'].min()
max_speed = filtered_data_2019_q3['avg_d_mbps'].max()

colormap = branca.colormap.linear.YlOrRd_09.scale(min_speed, max_speed)
colormap.caption = 'V. Internet (Mbps)'

# Obtenemos los colores para min y max
color_min = colormap(min_speed)
color_max = colormap(max_speed)

# Calculamos valores intermedios
v25 = min_speed + 0.25*(max_speed - min_speed)
v50 = min_speed + 0.50*(max_speed - min_speed)
v75 = min_speed + 0.75*(max_speed - min_speed)

# 3. Crear el mapa
mapa = folium.Map(location=centro, zoom_start=11, tiles="cartodbpositron")

# 4. Añadir las grillas con el borde del mismo color del fill
folium.GeoJson(
    data=filtered_data_2019_q3.to_json(),
    name="Grillas de Velocidad",
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['avg_d_mbps']),
        'color': colormap(feature['properties']['avg_d_mbps']),  # mismo color para el borde
        'weight': 0.5,
        'fillOpacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['avg_d_mbps', 'PROVINCIA'],
        aliases=['Velocidad (Mbps):', 'Provincia:']
    )
).add_to(mapa)

# Estilo para distritos, rojo para SAN MIGUEL y PUEBLO LIBRE, gris oscuro para el resto
def estilo_distritos(feature):
    distrito = feature['properties']['DISTRITO']
    if distrito in ["SAN MIGUEL", "PUEBLO LIBRE"]:
        return {
            'fillColor': 'none',
            'color': '#FF4500',
            'weight': 2,
            'fillOpacity': 0.0
        }
    else:
        return {
            'fillColor': 'none',
            'color': '#555555',
            'weight': 1,
            'fillOpacity': 0.0
        }

folium.GeoJson(
    data=lima_callao_gdf.to_json(),
    name="Distritos de Lima y Callao",
    style_function=estilo_distritos,
    tooltip=folium.GeoJsonTooltip(
        fields=['DISTRITO', 'PROVINCIA'],
        aliases=['Distrito:', 'Provincia:']
    )
).add_to(mapa)

# 5. Crear leyenda vertical personalizada con HTML y CSS, incluyendo valores intermedios
style = f"""
<style>
.mapLegend {{
  position: absolute;
  top: 400px;                /* Más abajo (antes 100px) */
  right: 125px;              /* Más a la izquierda (antes 50px) */
  background-color: rgba(255, 255, 255, 0.8);
  padding: 5px;              /* Menos padding (antes 10px) */
  border-radius: 5px;
  box-shadow: 0 0 5px rgba(0,0,0,0.3);
  z-index: 9999;
  text-align: center;
  font-family: sans-serif;
  font-size: 9px;            /* Texto más pequeño (antes sin especificar o default) */
}}
.mapLegend .legend-title {{
  font-weight: bold;
  margin-bottom: 5px;        /* Menos espacio debajo del título (antes 10px) */
}}
.mapLegend .legend-labels {{
  display: flex;
  flex-direction: column;
  align-items: center;
}}
.gradient-container {{
  position: relative;
  width: 15px;               /* Barra más delgada (antes 20px) */
  height: 120px;             /* Barra más baja (antes 150px) */
  margin: 5px 0;             /* Menos margen vertical (antes 10px 0) */
}}
.gradient-bar {{
  width: 15px;               /* Ajustar al nuevo ancho */
  height: 120px;             /* Ajustar a la nueva altura */
  background: linear-gradient(to top, {color_min} 0%, {color_max} 100%);
}}
.tick-label {{
  position: absolute;
  left: 20px;                /* Ajustar si quieres más cerca o lejos (antes 25px) */
  font-size: 9px;            /* Mismo tamaño que el contenedor o un poco menos */
  background-color: rgba(255,255,255,0.8);
  padding: 1px 2px;          /* Menos padding para texto */
  border-radius: 3px;
  white-space: nowrap;
  box-shadow: 0 0 3px rgba(0,0,0,0.2);
}}
.tick-label:after {{
  content: '';
  position: absolute;
  left: -5px;
  width: 5px;
  height: 1px;
  background: #000;
  top: 50%;
  transform: translateY(-50%);
}}
</style>
"""

mapa.get_root().html.add_child(Element(style))

legend_html = f"""
<div class='mapLegend'>
  <div class='legend-title'>{colormap.caption}</div>
  <div class='legend-labels'>
    <div class='max-label'>{max_speed:.2f} Mbps</div>
    <div class='gradient-container'>
      <div class='gradient-bar'></div>
      <div class='tick-label' style='top: 25%;'>{v75:.2f}</div>
      <div class='tick-label' style='top: 50%;'>{v50:.2f}</div>
      <div class='tick-label' style='top: 75%;'>{v25:.2f}</div>
    </div>
    <div class='min-label'>{min_speed:.2f} Mbps</div>
  </div>
</div>
"""

mapa.get_root().html.add_child(Element(legend_html))

folium.LayerControl().add_to(mapa)

mapa.save("mapa_lima_callao_con_grillas_colormap.html")
print("Mapa con leyenda vertical personalizada, distritos destacados y valores intermedios guardado con bordes de grillas del color de la velocidad.")



mapa

In [ ]:
import folium
import branca
import geopandas as gpd
from folium import Element
from pyproj import Transformer

# Suponiendo que lima_callao_gdf y filtered_data_2019_q3 están cargados en EPSG:4326

# 1. Reproyectar para calcular el centro
lima_callao_proj = lima_callao_gdf.to_crs(epsg=32718)
centroides = lima_callao_proj.geometry.centroid
centro_y = centroides.y.mean()
centro_x = centroides.x.mean()
transformer = Transformer.from_crs("EPSG:32718", "EPSG:4326", always_xy=True)
centro_lon, centro_lat = transformer.transform(centro_x, centro_y)
centro = [centro_lat, centro_lon]

# Si 'centroid' existe en filtered_data_2019_q3 la removemos
if 'centroid' in filtered_data_2019_q3.columns:
    filtered_data_2019_q3 = filtered_data_2019_q3.drop(columns='centroid')

# Extraemos las velocidades
speeds = filtered_data_2019_q3['avg_d_mbps']
min_speed = speeds.min()
max_speed = speeds.max()

# Calculamos cuantiles: Q0 (min), Q1(25%), Q2(50%), Q3(75%), Q4(100% max)
quantiles = speeds.quantile([0, 0.25, 0.5, 0.75, 1.0]).values
# quantiles[0] = min_speed, quantiles[1]=Q1, quantiles[2]=Q2, quantiles[3]=Q3, quantiles[4]=max_speed

# Creamos una escala de colores basada en cuantiles (StepColormap)
colors = ['#ffffcc','#ffeda0','#feb24c','#f03b20','#bd0026']  # Paleta YlOrRd en 5 pasos
colormap = branca.colormap.StepColormap(
    colors=colors,
    index=[quantiles[1], quantiles[2], quantiles[3]],  # Q1, Q2, Q3 como puntos de quiebre
    vmin=quantiles[0],
    vmax=quantiles[-1],
    caption='V. Internet (Mbps)'
)

# Cálculo de valores intermedios según esta distribución (opcional)
# Ahora, en vez de usar min, max lineal, si quieres mantener ticks en 25%, 50%, 75%
# en términos de velocidades reales:
v25 = quantiles[1]
v50 = quantiles[2]
v75 = quantiles[3]

# Crear el mapa
mapa = folium.Map(location=centro, zoom_start=11, tiles="cartodbpositron")

# Añadir las grillas con el borde del mismo color de relleno, según nueva escala
folium.GeoJson(
    data=filtered_data_2019_q3.to_json(),
    name="Grillas de Velocidad",
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['avg_d_mbps']),
        'color': colormap(feature['properties']['avg_d_mbps']),
        'weight': 0.5,
        'fillOpacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['avg_d_mbps', 'PROVINCIA'],
        aliases=['Velocidad (Mbps):', 'Provincia:']
    )
).add_to(mapa)

# Estilo para distritos
def estilo_distritos(feature):
    distrito = feature['properties']['DISTRITO']
    if distrito in ["SAN MIGUEL", "PUEBLO LIBRE"]:
        return {
            'fillColor': 'none',
            'color': '#FF4500',
            'weight': 2,
            'fillOpacity': 0.0
        }
    else:
        return {
            'fillColor': 'none',
            'color': '#555555',
            'weight': 1,
            'fillOpacity': 0.0
        }

folium.GeoJson(
    data=lima_callao_gdf.to_json(),
    name="Distritos de Lima y Callao",
    style_function=estilo_distritos,
    tooltip=folium.GeoJsonTooltip(
        fields=['DISTRITO', 'PROVINCIA'],
        aliases=['Distrito:', 'Provincia:']
    )
).add_to(mapa)

# Estilo CSS ajustado a tu gusto
style = f"""
<style>
.mapLegend {{
  position: absolute;
  top: 400px;
  right: 125px;
  background-color: rgba(255, 255, 255, 0.8);
  padding: 5px;
  border-radius: 5px;
  box-shadow: 0 0 5px rgba(0,0,0,0.3);
  z-index: 9999;
  text-align: center;
  font-family: sans-serif;
  font-size: 9px;
}}
.mapLegend .legend-title {{
  font-weight: bold;
  margin-bottom: 5px;
}}
.mapLegend .legend-labels {{
  display: flex;
  flex-direction: column;
  align-items: center;
}}
.gradient-container {{
  position: relative;
  width: 15px;
  height: 120px;
  margin: 5px 0;
}}
.gradient-bar {{
  width: 15px;
  height: 120px;
  /* Podemos usar un degradado igualado a los colores definidos o dejar así.
     Si quieres un gradiente continuo, puedes mantenerlo.
     Si quieres "pasos" marcados, necesitarás un CSS con steps,
     pero dado que usamos StepColormap, mejor dejar colormap.add_to(mapa) en lugar de la leyenda personalizada.
  */
  background: linear-gradient(to top, {colors[0]} 0%, {colors[1]} 25%, {colors[2]} 50%, {colors[3]} 75%, {colors[4]} 100%);
}}
.tick-label {{
  position: absolute;
  left: 20px;
  font-size: 9px;
  background-color: rgba(255,255,255,0.8);
  padding: 1px 2px;
  border-radius: 3px;
  white-space: nowrap;
  box-shadow: 0 0 3px rgba(0,0,0,0.2);
}}
.tick-label:after {{
  content: '';
  position: absolute;
  left: -5px;
  width: 5px;
  height: 1px;
  background: #000;
  top: 50%;
  transform: translateY(-50%);
}}
</style>
"""

mapa.get_root().html.add_child(Element(style))

legend_html = f"""
<div class='mapLegend'>
  <div class='legend-title'>{colormap.caption}</div>
  <div class='legend-labels'>
    <div class='max-label'>{quantiles[-1]:.2f} Mbps</div>
    <div class='gradient-container'>
      <div class='gradient-bar'></div>
      <div class='tick-label' style='top: 25%;'>{v75:.2f}</div>
      <div class='tick-label' style='top: 50%;'>{v50:.2f}</div>
      <div class='tick-label' style='top: 75%;'>{v25:.2f}</div>
    </div>
    <div class='min-label'>{quantiles[0]:.2f} Mbps</div>
  </div>
</div>
"""

mapa.get_root().html.add_child(Element(legend_html))

# Agregamos el control de capas
folium.LayerControl().add_to(mapa)

mapa.save("mapa_lima_callao_con_grillas_cuantiles.html")
print("Mapa con colormap basado en cuantiles guardado.")

In [ ]:
import folium
import branca
import geopandas as gpd
from folium import Element
from pyproj import Transformer

# Suponiendo que lima_callao_gdf y filtered_data_2019_q3 están en EPSG:4326

# 1. Reproyectar para calcular el centro
lima_callao_proj = lima_callao_gdf.to_crs(epsg=32718)
centroides = lima_callao_proj.geometry.centroid
centro_y = centroides.y.mean()
centro_x = centroides.x.mean()
transformer = Transformer.from_crs("EPSG:32718", "EPSG:4326", always_xy=True)
centro_lon, centro_lat = transformer.transform(centro_x, centro_y)
centro = [centro_lat, centro_lon]

# Si 'centroid' existe en filtered_data_2019_q3 la removemos
if 'centroid' in filtered_data_2019_q3.columns:
    filtered_data_2019_q3 = filtered_data_2019_q3.drop(columns='centroid')

speeds = filtered_data_2019_q3['avg_d_mbps']

# Definimos los límites de las categorías:
# Menor que 10 Mbps
# 10 a 25 Mbps
# 25 a 60 Mbps
# 60 a 100 Mbps
# 100 a 200 Mbps
# Mayor que 200 Mbps
colors = [
    '#ffffcc',  # <10
    '#ffeda0',  # 10-25
    '#fed976',  # 25-60
    '#fd8d3c',  # 60-100 (más oscuro)
    '#f03b20',  # 100-200 (más oscuro aún)
    '#bd0026'   # >200 (el más oscuro)
]

def color_por_categoria(speed):
    if speed > 200:
        return colors[5]
    elif speed > 100:
        return colors[4]
    elif speed > 60:
        return colors[3]
    elif speed > 25:
        return colors[2]
    elif speed > 10:
        return colors[1]
    else:
        return colors[0]

mapa = folium.Map(location=centro, zoom_start=11, tiles="cartodbpositron")

# Añadir las grillas
folium.GeoJson(
    data=filtered_data_2019_q3.to_json(),
    name="Grillas de Velocidad",
    style_function=lambda feature: {
        'fillColor': color_por_categoria(feature['properties']['avg_d_mbps']),
        'color': color_por_categoria(feature['properties']['avg_d_mbps']),
        'weight': 0.5,
        'fillOpacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['avg_d_mbps', 'PROVINCIA'],
        aliases=['Velocidad (Mbps):', 'Provincia:']
    )
).add_to(mapa)

# Estilo para distritos
def estilo_distritos(feature):
    distrito = feature['properties']['DISTRITO']
    if distrito in ["SAN MIGUEL", "PUEBLO LIBRE"]:
        return {
            'fillColor': 'none',
            'color': '#FF4500',
            'weight': 2,
            'fillOpacity': 0.0
        }
    else:
        return {
            'fillColor': 'none',
            'color': '#555555',
            'weight': 1,
            'fillOpacity': 0.0
        }

folium.GeoJson(
    data=lima_callao_gdf.to_json(),
    name="Distritos de Lima y Callao",
    style_function=estilo_distritos,
    tooltip=folium.GeoJsonTooltip(
        fields=['DISTRITO', 'PROVINCIA'],
        aliases=['Distrito:', 'Provincia:']
    )
).add_to(mapa)

# Estilo CSS para la leyenda
style = """
<style>
.mapLegend {
  position: absolute;
  top: 425px;
  right: 125px;
  background-color: rgba(255, 255, 255, 0.8);
  padding: 5px;
  border-radius: 5px;
  box-shadow: 0 0 5px rgba(0,0,0,0.3);
  z-index: 9999;
  text-align: left;
  font-family: sans-serif;
  font-size: 11px;
}
.mapLegend .legend-title {
  font-weight: bold;
  margin-bottom: 5px;
}
.color-box {
  display: inline-block;
  width: 12px;
  height: 12px;
  margin-right: 5px;
  vertical-align: middle;
}
.legend-item {
  margin-bottom: 3px;
}
</style>
"""

mapa.get_root().html.add_child(Element(style))

# Leyenda con las nuevas categorías
legend_html = f"""
<div class='mapLegend'>
  <div class='legend-title'>V. Internet (Mbps)</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[5]};'></span>Mayor que 200 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[4]};'></span>100 a 200 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[3]};'></span>60 a 100 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[2]};'></span>25 a 60 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[1]};'></span>10 a 25 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[0]};'></span>Menor que 10 Mbps</div>
</div>
"""

mapa.get_root().html.add_child(Element(legend_html))

folium.LayerControl().add_to(mapa)

mapa.save("mapa_lima_callao_categorias.html")
print("Mapa con categorías y colores más oscuros a partir de 60 Mbps guardado.")


In [ ]:
filtered_data_2019_q3 = quadkeys_final_final1[(quadkeys_final_final1['year'] == 2022) & (quadkeys_final_final1['quarter'] == 4)]

In [ ]:
import folium
import branca
import geopandas as gpd
from folium import Element
from pyproj import Transformer

# Suponiendo que lima_callao_gdf y filtered_data_2020_q3 están en EPSG:4326

# 1. Reproyectar para calcular el centro
lima_callao_proj = lima_callao_gdf.to_crs(epsg=32718)
centroides = lima_callao_proj.geometry.centroid
centro_y = centroides.y.mean()
centro_x = centroides.x.mean()
transformer = Transformer.from_crs("EPSG:32718", "EPSG:4326", always_xy=True)
centro_lon, centro_lat = transformer.transform(centro_x, centro_y)
centro = [centro_lat, centro_lon]

# Si 'centroid' existe en filtered_data_2020_q3 la removemos
if 'centroid' in filtered_data_2020_q3.columns:
    filtered_data_2020_q3 = filtered_data_2020_q3.drop(columns='centroid')

speeds = filtered_data_2020_q3['avg_d_mbps']

# Definimos los límites de las categorías
colors = [
    '#ffffcc',  # <10
    '#ffeda0',  # 10-25
    '#fed976',  # 25-60
    '#fd8d3c',  # 60-100
    '#f03b20',  # 100-200
    '#800000'   # >200
]

def color_por_categoria(speed):
    if speed > 200:
        return colors[5]
    elif speed > 100:
        return colors[4]
    elif speed > 60:
        return colors[3]
    elif speed > 25:
        return colors[2]
    elif speed > 10:
        return colors[1]
    else:
        return colors[0]

mapa = folium.Map(location=centro, zoom_start=11, tiles="cartodbpositron")

# Añadir las grillas
folium.GeoJson(
    data=filtered_data_2020_q3.to_json(),
    name="Grillas de Velocidad",
    style_function=lambda feature: {
        'fillColor': color_por_categoria(feature['properties']['avg_d_mbps']),
        'color': color_por_categoria(feature['properties']['avg_d_mbps']),
        'weight': 0.5,
        'fillOpacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['avg_d_mbps', 'PROVINCIA'],
        aliases=['Velocidad (Mbps):', 'Provincia:']
    )
).add_to(mapa)

# Color para SAN MIGUEL y PUEBLO LIBRE
line_color_special = '#67001F'

def estilo_distritos(feature):
    distrito = feature['properties']['DISTRITO']
    if distrito in ["SAN MIGUEL", "PUEBLO LIBRE"]:
        return {
            'fillColor': 'none',
            'color': line_color_special,
            'weight': 2,
            'fillOpacity': 0.0
        }
    else:
        return {
            'fillColor': 'none',
            'color': '#555555',
            'weight': 1,
            'fillOpacity': 0.0
        }

folium.GeoJson(
    data=lima_callao_gdf.to_json(),
    name="Distritos de Lima y Callao",
    style_function=estilo_distritos,
    tooltip=folium.GeoJsonTooltip(
        fields=['DISTRITO', 'PROVINCIA'],
        aliases=['Distrito:', 'Provincia:']
    )
).add_to(mapa)

# Estilo CSS usando f-string
style = f"""
<style>
.mapLegend {{
  position: absolute;
  top: 425px;
  right: 125px;
  background-color: rgba(255, 255, 255, 0.8);
  padding: 5px;
  border-radius: 5px;
  box-shadow: 0 0 5px rgba(0,0,0,0.3);
  z-index: 9999;
  font-family: sans-serif;
  font-size: 11px;
}}
.mapLegend .legend-title {{
  font-weight: bold;
  margin-bottom: 5px;
}}
.color-box {{
  display: inline-block;
  width: 12px;
  height: 12px;
  margin-right: 5px;
  vertical-align: middle;
}}
.legend-item {{
  margin-bottom: 3px;
}}
.line-box {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special};
}}
</style>
"""

mapa.get_root().html.add_child(Element(style))

# Leyenda con las categorías + línea especial para SAN MIGUEL y PUEBLO LIBRE
legend_html = f"""
<div class='mapLegend'>
  <div class='legend-title'>V. Internet (Mbps)</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[5]};'></span>Mayor que 200 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[4]};'></span>100 a 200 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[3]};'></span>60 a 100 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[2]};'></span>25 a 60 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[1]};'></span>10 a 25 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[0]};'></span>Menor que 10 Mbps</div>
  <hr style="margin:5px 0;">
  <div class='legend-item'><span class='line-box'></span>2019Q4</div>
</div>
"""

mapa.get_root().html.add_child(Element(legend_html))

folium.LayerControl().add_to(mapa)

mapa.save("mapa_lima_callao_categorias.html")
print("Mapa guardado con línea para SAN MIGUEL y PUEBLO LIBRE en la leyenda.")


In [ ]:
import folium
import branca
import geopandas as gpd
from folium import Element
from pyproj import Transformer

# Suponiendo que lima_callao_gdf y filtered_data_2019_q3 están en EPSG:4326

# 1. Reproyectar para calcular el centro
lima_callao_proj = lima_callao_gdf.to_crs(epsg=32718)
centroides = lima_callao_proj.geometry.centroid
centro_y = centroides.y.mean()
centro_x = centroides.x.mean()
transformer = Transformer.from_crs("EPSG:32718", "EPSG:4326", always_xy=True)
centro_lon, centro_lat = transformer.transform(centro_x, centro_y)
centro = [centro_lat, centro_lon]

# Si 'centroid' existe en filtered_data_2019_q3 la removemos
if 'centroid' in filtered_data_2019_q3.columns:
    filtered_data_2020_q3 = filtered_data_2020_q3.drop(columns='centroid')

speeds = filtered_data_2020_q3['avg_d_mbps']

# Definimos los límites de las categorías:
# Menor que 10 Mbps
# 10 a 25 Mbps
# 25 a 60 Mbps
# 60 a 100 Mbps
# 100 a 200 Mbps
# Mayor que 200 Mbps
colors = [
    '#ffffcc',  # <10
    '#ffeda0',  # 10-25
    '#fed976',  # 25-60
    '#fd8d3c',  # 60-100 (más oscuro)
    '#f03b20',  # 100-200 (más oscuro aún)
    '#800000'   # >200 (el más oscuro)
]

def color_por_categoria(speed):
    if speed > 200:
        return colors[5]
    elif speed > 100:
        return colors[4]
    elif speed > 60:
        return colors[3]
    elif speed > 25:
        return colors[2]
    elif speed > 10:
        return colors[1]
    else:
        return colors[0]

mapa = folium.Map(location=centro, zoom_start=11, tiles="cartodbpositron")

# Añadir las grillas
folium.GeoJson(
    data=filtered_data_2020_q3.to_json(),
    name="Grillas de Velocidad",
    style_function=lambda feature: {
        'fillColor': color_por_categoria(feature['properties']['avg_d_mbps']),
        'color': color_por_categoria(feature['properties']['avg_d_mbps']),
        'weight': 0.5,
        'fillOpacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['avg_d_mbps', 'PROVINCIA'],
        aliases=['Velocidad (Mbps):', 'Provincia:']
    )
).add_to(mapa)

# Estilo para distritos
def estilo_distritos(feature):
    distrito = feature['properties']['DISTRITO']
    if distrito in ["SAN MIGUEL", "PUEBLO LIBRE"]:
        return {
            'fillColor': 'none',
            'color': '#67001F',
            'weight': 2,
            'fillOpacity': 0.0
        }
    else:
        return {
            'fillColor': 'none',
            'color': '#555555',
            'weight': 1,
            'fillOpacity': 0.0
        }

folium.GeoJson(
    data=lima_callao_gdf.to_json(),
    name="Distritos de Lima y Callao",
    style_function=estilo_distritos,
    tooltip=folium.GeoJsonTooltip(
        fields=['DISTRITO', 'PROVINCIA'],
        aliases=['Distrito:', 'Provincia:']
    )
).add_to(mapa)

# Estilo CSS para la leyenda
style = """
<style>
.mapLegend {
  position: absolute;
  top: 425px;
  right: 125px;
  background-color: rgba(255, 255, 255, 0.8);
  padding: 5px;
  border-radius: 5px;
  box-shadow: 0 0 5px rgba(0,0,0,0.3);
  z-index: 9999;
  text-align: left;
  font-family: sans-serif;
  font-size: 11px;
}
.mapLegend .legend-title {
  font-weight: bold;
  margin-bottom: 5px;
}
.color-box {
  display: inline-block;
  width: 12px;
  height: 12px;
  margin-right: 5px;
  vertical-align: middle;
}
.legend-item {
  margin-bottom: 3px;
}
</style>
"""

mapa.get_root().html.add_child(Element(style))

# Leyenda con las nuevas categorías
legend_html = f"""
<div class='mapLegend'>
  <div class='legend-title'>V. Internet (Mbps)</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[5]};'></span>Mayor que 200 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[4]};'></span>100 a 200 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[3]};'></span>60 a 100 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[2]};'></span>25 a 60 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[1]};'></span>10 a 25 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[0]};'></span>Menor que 10 Mbps</div>
</div>
"""

mapa.get_root().html.add_child(Element(legend_html))

folium.LayerControl().add_to(mapa)

mapa.save("mapa_lima_callao_categorias.html")
print("Mapa con categorías y colores más oscuros a partir de 60 Mbps guardado.")

In [ ]:
import folium
import branca
import geopandas as gpd
from folium import Element
from pyproj import Transformer

# Suponiendo que lima_callao_gdf y filtered_data_2020_q3 están en EPSG:4326

# 1. Reproyectar para calcular el centro
lima_callao_proj = lima_callao_gdf.to_crs(epsg=32718)
centroides = lima_callao_proj.geometry.centroid
centro_y = centroides.y.mean()
centro_x = centroides.x.mean()
transformer = Transformer.from_crs("EPSG:32718", "EPSG:4326", always_xy=True)
centro_lon, centro_lat = transformer.transform(centro_x, centro_y)
centro = [centro_lat, centro_lon]

# Si 'centroid' existe en filtered_data_2020_q3 la removemos
if 'centroid' in filtered_data_2020_q3.columns:
    filtered_data_2020_q3 = filtered_data_2020_q3.drop(columns='centroid')

speeds = filtered_data_2020_q3['avg_d_mbps']

# Definimos los límites de las categorías (iguales que antes)
colors = [
    '#ffffcc',  # <10
    '#ffeda0',  # 10-25
    '#fed976',  # 25-60
    '#fd8d3c',  # 60-100
    '#f03b20',  # 100-200
    '#800000'   # >200
]

def color_por_categoria(speed):
    if speed > 200:
        return colors[5]
    elif speed > 100:
        return colors[4]
    elif speed > 60:
        return colors[3]
    elif speed > 25:
        return colors[2]
    elif speed > 10:
        return colors[1]
    else:
        return colors[0]

mapa = folium.Map(location=centro, zoom_start=11, tiles="cartodbpositron")

# Añadir las grillas
folium.GeoJson(
    data=filtered_data_2020_q3.to_json(),
    name="Grillas de Velocidad",
    style_function=lambda feature: {
        'fillColor': color_por_categoria(feature['properties']['avg_d_mbps']),
        'color': color_por_categoria(feature['properties']['avg_d_mbps']),
        'weight': 0.5,
        'fillOpacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['avg_d_mbps', 'PROVINCIA'],
        aliases=['Velocidad (Mbps):', 'Provincia:']
    )
).add_to(mapa)

# Colores para los distintos grupos de distritos
line_color_special = '#67001F'   # SAN MIGUEL y PUEBLO LIBRE
line_color_special_2 = '#7A0177' # SURQUILLO, MAGDALENA DEL MAR, JESUS MARIA, LINCE, BREÑA

def estilo_distritos(feature):
    distrito = feature['properties']['DISTRITO']
    if distrito in ["SAN MIGUEL", "PUEBLO LIBRE"]:
        return {
            'fillColor': 'none',
            'color': line_color_special,
            'weight': 2,
            'fillOpacity': 0.0
        }
    elif distrito in ["SURQUILLO", "MAGDALENA DEL MAR", "JESUS MARIA", "LINCE", "BREÑA"]:
        return {
            'fillColor': 'none',
            'color': line_color_special_2,
            'weight': 2,
            'fillOpacity': 0.0
        }
    else:
        return {
            'fillColor': 'none',
            'color': '#555555',
            'weight': 1,
            'fillOpacity': 0.0
        }

folium.GeoJson(
    data=lima_callao_gdf.to_json(),
    name="Distritos de Lima y Callao",
    style_function=estilo_distritos,
    tooltip=folium.GeoJsonTooltip(
        fields=['DISTRITO', 'PROVINCIA'],
        aliases=['Distrito:', 'Provincia:']
    )
).add_to(mapa)

# Estilo CSS usando f-string
style = f"""
<style>
.mapLegend {{
  position: absolute;
  top: 425px;
  right: 125px;
  background-color: rgba(255, 255, 255, 0.8);
  padding: 5px;
  border-radius: 5px;
  box-shadow: 0 0 5px rgba(0,0,0,0.3);
  z-index: 9999;
  font-family: sans-serif;
  font-size: 11px;
}}
.mapLegend .legend-title {{
  font-weight: bold;
  margin-bottom: 5px;
}}
.color-box {{
  display: inline-block;
  width: 12px;
  height: 12px;
  margin-right: 5px;
  vertical-align: middle;
}}
.legend-item {{
  margin-bottom: 3px;
}}
.line-box {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special};
}}
.line-box-2 {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special_2};
}}
</style>
"""

mapa.get_root().html.add_child(Element(style))

# Leyenda con las categorías + líneas especiales
# Primero las categorías de velocidad
# Luego, debajo de una línea horizontal, indicamos 2019Q4 (primer grupo) y 2020Q3 (segundo grupo)
legend_html = f"""
<div class='mapLegend'>
  <div class='legend-title'>V. Internet (Mbps)</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[5]};'></span>Mayor que 200 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[4]};'></span>100 a 200 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[3]};'></span>60 a 100 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[2]};'></span>25 a 60 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[1]};'></span>10 a 25 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[0]};'></span>Menor que 10 Mbps</div>
  <hr style="margin:5px 0;">
  <div class='legend-item'><span class='line-box'></span>2019Q4</div>
  <div class='legend-item'><span class='line-box-2'></span>2020Q3</div>
</div>
"""

mapa.get_root().html.add_child(Element(legend_html))

folium.LayerControl().add_to(mapa)

mapa.save("mapa_lima_callao_categorias.html")
print("Mapa guardado con línea para SAN MIGUEL, PUEBLO LIBRE (2019Q4) y SURQUILLO, MAGDALENA DEL MAR, JESUS MARIA, LINCE, BREÑA (2020Q3) en la leyenda.")


In [ ]:
import folium
import branca
import geopandas as gpd
from folium import Element
from pyproj import Transformer

# Suponiendo que lima_callao_gdf y filtered_data_2020_q3 están en EPSG:4326

# 1. Reproyectar para calcular el centro
lima_callao_proj = lima_callao_gdf.to_crs(epsg=32718)
centroides = lima_callao_proj.geometry.centroid
centro_y = centroides.y.mean()
centro_x = centroides.x.mean()
transformer = Transformer.from_crs("EPSG:32718", "EPSG:4326", always_xy=True)
centro_lon, centro_lat = transformer.transform(centro_x, centro_y)
centro = [centro_lat, centro_lon]

# Si 'centroid' existe en filtered_data_2020_q3 la removemos
if 'centroid' in filtered_data_2020_q3.columns:
    filtered_data_2020_q3 = filtered_data_2020_q3.drop(columns='centroid')

speeds = filtered_data_2020_q3['avg_d_mbps']

# Definimos los límites de las categorías (iguales que antes)
colors = [
    '#ffffcc',  # <10
    '#ffeda0',  # 10-25
    '#fed976',  # 25-60
    '#fd8d3c',  # 60-100
    '#f03b20',  # 100-200
    '#800000'   # >200
]

def color_por_categoria(speed):
    if speed > 200:
        return colors[5]
    elif speed > 100:
        return colors[4]
    elif speed > 60:
        return colors[3]
    elif speed > 25:
        return colors[2]
    elif speed > 10:
        return colors[1]
    else:
        return colors[0]

mapa = folium.Map(location=centro, zoom_start=11, tiles="cartodbpositron")

# Añadir las grillas
folium.GeoJson(
    data=filtered_data_2020_q3.to_json(),
    name="Grillas de Velocidad",
    style_function=lambda feature: {
        'fillColor': color_por_categoria(feature['properties']['avg_d_mbps']),
        'color': color_por_categoria(feature['properties']['avg_d_mbps']),
        'weight': 0.5,
        'fillOpacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['avg_d_mbps', 'PROVINCIA'],
        aliases=['Velocidad (Mbps):', 'Provincia:']
    )
).add_to(mapa)

# Colores para los distintos grupos de distritos
line_color_special = '#67001F'   # SAN MIGUEL y PUEBLO LIBRE
line_color_special_2 = '#980043' # SURQUILLO, MAGDALENA DEL MAR, JESUS MARIA, LINCE, BREÑA
line_color_special_3 = '#6A51A3' # SANTIAGO DE SURCO, MIRAFLORES, CHORRILLOS, LIMA, etc.

def estilo_distritos(feature):
    distrito = feature['properties']['DISTRITO']
    if distrito in ["SAN MIGUEL", "PUEBLO LIBRE"]:
        return {
            'fillColor': 'none',
            'color': line_color_special,
            'weight': 2,
            'fillOpacity': 0.0
        }
    elif distrito in ["SURQUILLO", "MAGDALENA DEL MAR", "JESUS MARIA", "LINCE", "BREÑA"]:
        return {
            'fillColor': 'none',
            'color': line_color_special_2,
            'weight': 2,
            'fillOpacity': 0.0
        }
    elif distrito in ["SANTIAGO DE SURCO", "MIRAFLORES", "CHORRILLOS", "LIMA",
                      "SAN ISIDRO", "CALLAO", "LOS OLIVOS", "LA MOLINA",
                      "EL AGUSTINO", "SAN MARTIN DE PORRES", "SAN JUAN DE LURIGANCHO"]:
        return {
            'fillColor': 'none',
            'color': line_color_special_3,
            'weight': 2,
            'fillOpacity': 0.0
        }
    else:
        return {
            'fillColor': 'none',
            'color': '#555555',
            'weight': 1,
            'fillOpacity': 0.0
        }

folium.GeoJson(
    data=lima_callao_gdf.to_json(),
    name="Distritos de Lima y Callao",
    style_function=estilo_distritos,
    tooltip=folium.GeoJsonTooltip(
        fields=['DISTRITO', 'PROVINCIA'],
        aliases=['Distrito:', 'Provincia:']
    )
).add_to(mapa)

# Estilo CSS usando f-string
style = f"""
<style>
.mapLegend {{
  position: absolute;
  top: 425px;
  right: 125px;
  background-color: rgba(255, 255, 255, 0.8);
  padding: 5px;
  border-radius: 5px;
  box-shadow: 0 0 5px rgba(0,0,0,0.3);
  z-index: 9999;
  font-family: sans-serif;
  font-size: 11px;
}}
.mapLegend .legend-title {{
  font-weight: bold;
  margin-bottom: 5px;
}}
.color-box {{
  display: inline-block;
  width: 12px;
  height: 12px;
  margin-right: 5px;
  vertical-align: middle;
}}
.legend-item {{
  margin-bottom: 3px;
}}
.line-box {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special};
}}
.line-box-2 {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special_2};
}}
.line-box-3 {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special_3};
}}
</style>
"""

mapa.get_root().html.add_child(Element(style))

# Leyenda con las categorías + líneas especiales
legend_html = f"""
<div class='mapLegend'>
  <div class='legend-title'>V. Internet (Mbps)</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[5]};'></span>Mayor que 200 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[4]};'></span>100 a 200 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[3]};'></span>60 a 100 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[2]};'></span>25 a 60 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[1]};'></span>10 a 25 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[0]};'></span>Menor que 10 Mbps</div>
  <hr style="margin:5px 0;">
  <div class='legend-item'><span class='line-box'></span>2019Q4</div>
  <div class='legend-item'><span class='line-box-2'></span>2020Q3</div>
  <div class='legend-item'><span class='line-box-3'></span>2020Q4</div>
</div>
"""

mapa.get_root().html.add_child(Element(legend_html))

folium.LayerControl().add_to(mapa)

mapa.save("mapa_lima_callao_categorias.html")
print("Mapa guardado con líneas para 2019Q4, 2020Q3 y 2020Q4 en la leyenda.")


In [ ]:
import folium
import branca
import geopandas as gpd
from folium import Element
from pyproj import Transformer

# Suponiendo que lima_callao_gdf y filtered_data_2020_q3 están en EPSG:4326

# 1. Reproyectar para calcular el centro
lima_callao_proj = lima_callao_gdf.to_crs(epsg=32718)
centroides = lima_callao_proj.geometry.centroid
centro_y = centroides.y.mean()
centro_x = centroides.x.mean()
transformer = Transformer.from_crs("EPSG:32718", "EPSG:4326", always_xy=True)
centro_lon, centro_lat = transformer.transform(centro_x, centro_y)
centro = [centro_lat, centro_lon]

# Si 'centroid' existe en filtered_data_2020_q3 la removemos
if 'centroid' in filtered_data_2020_q3.columns:
    filtered_data_2020_q3 = filtered_data_2020_q3.drop(columns='centroid')

speeds = filtered_data_2020_q3['avg_d_mbps']

# Definimos los límites de las categorías
colors = [
    '#ffffcc',  # <10
    '#ffeda0',  # 10-25
    '#fed976',  # 25-60
    '#fd8d3c',  # 60-100
    '#f03b20',  # 100-200
    '#800000'   # >200
]

def color_por_categoria(speed):
    if speed > 200:
        return colors[5]
    elif speed > 100:
        return colors[4]
    elif speed > 60:
        return colors[3]
    elif speed > 25:
        return colors[2]
    elif speed > 10:
        return colors[1]
    else:
        return colors[0]

mapa = folium.Map(location=centro, zoom_start=11, tiles="cartodbpositron")

# Añadir las grillas
folium.GeoJson(
    data=filtered_data_2020_q3.to_json(),
    name="Grillas de Velocidad",
    style_function=lambda feature: {
        'fillColor': color_por_categoria(feature['properties']['avg_d_mbps']),
        'color': color_por_categoria(feature['properties']['avg_d_mbps']),
        'weight': 0.5,
        'fillOpacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['avg_d_mbps', 'PROVINCIA'],
        aliases=['Velocidad (Mbps):', 'Provincia:']
    )
).add_to(mapa)

# Colores para los distintos grupos de distritos
line_color_special = '#67001F'   # SAN MIGUEL y PUEBLO LIBRE
line_color_special_2 = '#980043' # SURQUILLO, MAGDALENA DEL MAR, JESUS MARIA, LINCE, BREÑA
line_color_special_3 = '#6A51A3' # SANTIAGO DE SURCO, MIRAFLORES, etc.
line_color_special_4 = '#045A8D' # LA VICTORIA, SAN LUIS, BARRANCO

def estilo_distritos(feature):
    distrito = feature['properties']['DISTRITO']
    if distrito in ["SAN MIGUEL", "PUEBLO LIBRE"]:
        return {
            'fillColor': 'none',
            'color': line_color_special,
            'weight': 2,
            'fillOpacity': 0.0
        }
    elif distrito in ["SURQUILLO", "MAGDALENA DEL MAR", "JESUS MARIA", "LINCE", "BREÑA"]:
        return {
            'fillColor': 'none',
            'color': line_color_special_2,
            'weight': 2,
            'fillOpacity': 0.0
        }
    elif distrito in ["SANTIAGO DE SURCO", "MIRAFLORES", "CHORRILLOS", "LIMA",
                      "SAN ISIDRO", "CALLAO", "LOS OLIVOS", "LA MOLINA",
                      "EL AGUSTINO", "SAN MARTIN DE PORRES", "SAN JUAN DE LURIGANCHO"]:
        return {
            'fillColor': 'none',
            'color': line_color_special_3,
            'weight': 2,
            'fillOpacity': 0.0
        }
    elif distrito in ["LA VICTORIA", "SAN LUIS", "BARRANCO"]:
        return {
            'fillColor': 'none',
            'color': line_color_special_4,
            'weight': 2,
            'fillOpacity': 0.0
        }
    else:
        return {
            'fillColor': 'none',
            'color': '#555555',
            'weight': 1,
            'fillOpacity': 0.0
        }

folium.GeoJson(
    data=lima_callao_gdf.to_json(),
    name="Distritos de Lima y Callao",
    style_function=estilo_distritos,
    tooltip=folium.GeoJsonTooltip(
        fields=['DISTRITO', 'PROVINCIA'],
        aliases=['Distrito:', 'Provincia:']
    )
).add_to(mapa)

# Estilo CSS usando f-string
style = f"""
<style>
.mapLegend {{
  position: absolute;
  top: 385px;
  right: 125px;
  background-color: rgba(255, 255, 255, 0.8);
  padding: 5px;
  border-radius: 5px;
  box-shadow: 0 0 5px rgba(0,0,0,0.3);
  z-index: 9999;
  font-family: sans-serif;
  font-size: 11px;
}}
.mapLegend .legend-title {{
  font-weight: bold;
  margin-bottom: 5px;
}}
.color-box {{
  display: inline-block;
  width: 12px;
  height: 12px;
  margin-right: 5px;
  vertical-align: middle;
}}
.legend-item {{
  margin-bottom: 3px;
}}
.line-box {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special};
}}
.line-box-2 {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special_2};
}}
.line-box-3 {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special_3};
}}
.line-box-4 {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special_4};
}}
</style>
"""

mapa.get_root().html.add_child(Element(style))

# Leyenda con las categorías + líneas especiales
legend_html = f"""
<div class='mapLegend'>
  <div class='legend-title'>V. Internet (Mbps)</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[5]};'></span>Mayor que 200 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[4]};'></span>100 a 200 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[3]};'></span>60 a 100 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[2]};'></span>25 a 60 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[1]};'></span>10 a 25 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[0]};'></span>Menor que 10 Mbps</div>
  <hr style="margin:5px 0;">
  <div class='legend-item'><span class='line-box'></span>2019Q4</div>
  <div class='legend-item'><span class='line-box-2'></span>2020Q3</div>
  <div class='legend-item'><span class='line-box-3'></span>2020Q4</div>
  <div class='legend-item'><span class='line-box-4'></span>2021Q1</div>
</div>
"""

mapa.get_root().html.add_child(Element(legend_html))

folium.LayerControl().add_to(mapa)

mapa.save("mapa_lima_callao_categorias.html")
print("Mapa guardado con líneas para 2019Q4, 2020Q3, 2020Q4 y 2021Q1 en la leyenda.")


In [ ]:
import folium
import branca
import geopandas as gpd
from folium import Element
from pyproj import Transformer

# Suponiendo que lima_callao_gdf y filtered_data_2020_q3 están en EPSG:4326

# 1. Reproyectar para calcular el centro
lima_callao_proj = lima_callao_gdf.to_crs(epsg=32718)
centroides = lima_callao_proj.geometry.centroid
centro_y = centroides.y.mean()
centro_x = centroides.x.mean()
transformer = Transformer.from_crs("EPSG:32718", "EPSG:4326", always_xy=True)
centro_lon, centro_lat = transformer.transform(centro_x, centro_y)
centro = [centro_lat, centro_lon]

# Si 'centroid' existe en filtered_data_2020_q3 la removemos
if 'centroid' in filtered_data_2020_q3.columns:
    filtered_data_2020_q3 = filtered_data_2020_q3.drop(columns='centroid')

speeds = filtered_data_2020_q3['avg_d_mbps']

# Definimos los límites de las categorías
colors = [
    '#ffffcc',  # <10
    '#ffeda0',  # 10-25
    '#fed976',  # 25-60
    '#fd8d3c',  # 60-100
    '#f03b20',  # 100-200
    '#800000'   # >200
]

def color_por_categoria(speed):
    if speed > 200:
        return colors[5]
    elif speed > 100:
        return colors[4]
    elif speed > 60:
        return colors[3]
    elif speed > 25:
        return colors[2]
    elif speed > 10:
        return colors[1]
    else:
        return colors[0]

mapa = folium.Map(location=centro, zoom_start=11, tiles="cartodbpositron")

# Añadir las grillas
folium.GeoJson(
    data=filtered_data_2020_q3.to_json(),
    name="Grillas de Velocidad",
    style_function=lambda feature: {
        'fillColor': color_por_categoria(feature['properties']['avg_d_mbps']),
        'color': color_por_categoria(feature['properties']['avg_d_mbps']),
        'weight': 0.5,
        'fillOpacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['avg_d_mbps', 'PROVINCIA'],
        aliases=['Velocidad (Mbps):', 'Provincia:']
    )
).add_to(mapa)

# Colores para los distintos grupos de distritos
line_color_special = '#67001F'   # SAN MIGUEL y PUEBLO LIBRE
line_color_special_2 = '#980043' # SURQUILLO, MAGDALENA DEL MAR, JESUS MARIA, LINCE, BREÑA
line_color_special_3 = '#6A51A3' # SANTIAGO DE SURCO, MIRAFLORES, etc.
line_color_special_4 = '#045A8D' # LA VICTORIA, SAN LUIS, BARRANCO
line_color_special_5 = '#016C59' # VILLA MARIA DEL TRIUNFO, etc.

def estilo_distritos(feature):
    distrito = feature['properties']['DISTRITO']
    if distrito in ["SAN MIGUEL", "PUEBLO LIBRE"]:
        return {
            'fillColor': 'none',
            'color': line_color_special,
            'weight': 2,
            'fillOpacity': 0.0
        }
    elif distrito in ["SURQUILLO", "MAGDALENA DEL MAR", "JESUS MARIA", "LINCE", "BREÑA"]:
        return {
            'fillColor': 'none',
            'color': line_color_special_2,
            'weight': 2,
            'fillOpacity': 0.0
        }
    elif distrito in ["SANTIAGO DE SURCO", "MIRAFLORES", "CHORRILLOS", "LIMA",
                      "SAN ISIDRO", "CALLAO", "LOS OLIVOS", "LA MOLINA",
                      "EL AGUSTINO", "SAN MARTIN DE PORRES", "SAN JUAN DE LURIGANCHO"]:
        return {
            'fillColor': 'none',
            'color': line_color_special_3,
            'weight': 2,
            'fillOpacity': 0.0
        }
    elif distrito in ["LA VICTORIA", "SAN LUIS", "BARRANCO"]:
        return {
            'fillColor': 'none',
            'color': line_color_special_4,
            'weight': 2,
            'fillOpacity': 0.0
        }
    elif distrito in ["VILLA MARIA DEL TRIUNFO", "VILLA EL SALVADOR", "INDEPENDENCIA",
                      "CARMEN DE LA LEGUA REYNOSO", "ATE", "SAN JUAN DE MIRAFLORES"]:
        return {
            'fillColor': 'none',
            'color': line_color_special_5,
            'weight': 2,
            'fillOpacity': 0.0
        }
    else:
        return {
            'fillColor': 'none',
            'color': '#555555',
            'weight': 1,
            'fillOpacity': 0.0
        }

folium.GeoJson(
    data=lima_callao_gdf.to_json(),
    name="Distritos de Lima y Callao",
    style_function=estilo_distritos,
    tooltip=folium.GeoJsonTooltip(
        fields=['DISTRITO', 'PROVINCIA'],
        aliases=['Distrito:', 'Provincia:']
    )
).add_to(mapa)

# Estilo CSS usando f-string
style = f"""
<style>
.mapLegend {{
  position: absolute;
  top: 375px;
  right: 125px;
  background-color: rgba(255, 255, 255, 0.8);
  padding: 5px;
  border-radius: 5px;
  box-shadow: 0 0 5px rgba(0,0,0,0.3);
  z-index: 9999;
  font-family: sans-serif;
  font-size: 11px;
}}
.mapLegend .legend-title {{
  font-weight: bold;
  margin-bottom: 5px;
}}
.color-box {{
  display: inline-block;
  width: 12px;
  height: 12px;
  margin-right: 5px;
  vertical-align: middle;
}}
.legend-item {{
  margin-bottom: 3px;
}}
.line-box {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special};
}}
.line-box-2 {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special_2};
}}
.line-box-3 {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special_3};
}}
.line-box-4 {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special_4};
}}
.line-box-5 {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special_5};
}}
</style>
"""

mapa.get_root().html.add_child(Element(style))

# Leyenda con las categorías + líneas especiales
legend_html = f"""
<div class='mapLegend'>
  <div class='legend-title'>V. Internet (Mbps)</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[5]};'></span>Mayor que 200 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[4]};'></span>100 a 200 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[3]};'></span>60 a 100 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[2]};'></span>25 a 60 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[1]};'></span>10 a 25 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[0]};'></span>Menor que 10 Mbps</div>
  <hr style="margin:5px 0;">
  <div class='legend-item'><span class='line-box'></span>2019Q4</div>
  <div class='legend-item'><span class='line-box-2'></span>2020Q3</div>
  <div class='legend-item'><span class='line-box-3'></span>2020Q4</div>
  <div class='legend-item'><span class='line-box-4'></span>2021Q1</div>
  <div class='legend-item'><span class='line-box-5'></span>2021Q2</div>
</div>
"""

mapa.get_root().html.add_child(Element(legend_html))

folium.LayerControl().add_to(mapa)

mapa.save("mapa_lima_callao_categorias.html")
print("Mapa guardado con líneas para 2019Q4, 2020Q3, 2020Q4, 2021Q1 y 2021Q2 en la leyenda.")

In [ ]:
import folium
import branca
import geopandas as gpd
from folium import Element
from pyproj import Transformer

# Suponiendo que lima_callao_gdf y filtered_data_2020_q3 están en EPSG:4326

# 1. Reproyectar para calcular el centro
lima_callao_proj = lima_callao_gdf.to_crs(epsg=32718)
centroides = lima_callao_proj.geometry.centroid
centro_y = centroides.y.mean()
centro_x = centroides.x.mean()
transformer = Transformer.from_crs("EPSG:32718", "EPSG:4326", always_xy=True)
centro_lon, centro_lat = transformer.transform(centro_x, centro_y)
centro = [centro_lat, centro_lon]

# Si 'centroid' existe en filtered_data_2020_q3 la removemos
if 'centroid' in filtered_data_2020_q3.columns:
    filtered_data_2020_q3 = filtered_data_2020_q3.drop(columns='centroid')

speeds = filtered_data_2020_q3['avg_d_mbps']

# Definimos los límites de las categorías
colors = [
    '#ffffcc',  # <10
    '#ffeda0',  # 10-25
    '#fed976',  # 25-60
    '#fd8d3c',  # 60-100
    '#f03b20',  # 100-200
    '#800000'   # >200
]

def color_por_categoria(speed):
    if speed > 200:
        return colors[5]
    elif speed > 100:
        return colors[4]
    elif speed > 60:
        return colors[3]
    elif speed > 25:
        return colors[2]
    elif speed > 10:
        return colors[1]
    else:
        return colors[0]

mapa = folium.Map(location=centro, zoom_start=11, tiles="cartodbpositron")

# Añadir las grillas
folium.GeoJson(
    data=filtered_data_2020_q3.to_json(),
    name="Grillas de Velocidad",
    style_function=lambda feature: {
        'fillColor': color_por_categoria(feature['properties']['avg_d_mbps']),
        'color': color_por_categoria(feature['properties']['avg_d_mbps']),
        'weight': 0.5,
        'fillOpacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['avg_d_mbps', 'PROVINCIA'],
        aliases=['Velocidad (Mbps):', 'Provincia:']
    )
).add_to(mapa)

# Colores para los distintos grupos de distritos
line_color_special = '#67001F'   # SAN MIGUEL y PUEBLO LIBRE
line_color_special_2 = '#980043' # SURQUILLO, MAGDALENA DEL MAR, JESUS MARIA, LINCE, BREÑA
line_color_special_3 = '#6A51A3' # SANTIAGO DE SURCO, MIRAFLORES, etc.
line_color_special_4 = '#045A8D' # LA VICTORIA, SAN LUIS, BARRANCO
line_color_special_5 = '#016C59' # VILLA MARIA DEL TRIUNFO, etc.
line_color_special_6 = '#C51B7D' # COMAS, RIMAC, SANTA ANITA, SAN BORJA, CARABAYLLO

def estilo_distritos(feature):
    distrito = feature['properties']['DISTRITO']
    if distrito in ["SAN MIGUEL", "PUEBLO LIBRE"]:
        return {
            'fillColor': 'none',
            'color': line_color_special,
            'weight': 2,
            'fillOpacity': 0.0
        }
    elif distrito in ["SURQUILLO", "MAGDALENA DEL MAR", "JESUS MARIA", "LINCE", "BREÑA"]:
        return {
            'fillColor': 'none',
            'color': line_color_special_2,
            'weight': 2,
            'fillOpacity': 0.0
        }
    elif distrito in ["SANTIAGO DE SURCO", "MIRAFLORES", "CHORRILLOS", "LIMA",
                      "SAN ISIDRO", "CALLAO", "LOS OLIVOS", "LA MOLINA",
                      "EL AGUSTINO", "SAN MARTIN DE PORRES", "SAN JUAN DE LURIGANCHO"]:
        return {
            'fillColor': 'none',
            'color': line_color_special_3,
            'weight': 2,
            'fillOpacity': 0.0
        }
    elif distrito in ["LA VICTORIA", "SAN LUIS", "BARRANCO"]:
        return {
            'fillColor': 'none',
            'color': line_color_special_4,
            'weight': 2,
            'fillOpacity': 0.0
        }
    elif distrito in ["VILLA MARIA DEL TRIUNFO", "VILLA EL SALVADOR", "INDEPENDENCIA",
                      "CARMEN DE LA LEGUA REYNOSO", "ATE", "SAN JUAN DE MIRAFLORES"]:
        return {
            'fillColor': 'none',
            'color': line_color_special_5,
            'weight': 2,
            'fillOpacity': 0.0
        }
    elif distrito in ["COMAS", "RIMAC", "SANTA ANITA", "SAN BORJA", "CARABAYLLO"]:
        return {
            'fillColor': 'none',
            'color': line_color_special_6,
            'weight': 2,
            'fillOpacity': 0.0
        }
    else:
        return {
            'fillColor': 'none',
            'color': '#555555',
            'weight': 1,
            'fillOpacity': 0.0
        }

folium.GeoJson(
    data=lima_callao_gdf.to_json(),
    name="Distritos de Lima y Callao",
    style_function=estilo_distritos,
    tooltip=folium.GeoJsonTooltip(
        fields=['DISTRITO', 'PROVINCIA'],
        aliases=['Distrito:', 'Provincia:']
    )
).add_to(mapa)

# Estilo CSS usando f-string
style = f"""
<style>
.mapLegend {{
  position: absolute;
  top: 350px;
  right: 125px;
  background-color: rgba(255, 255, 255, 0.8);
  padding: 5px;
  border-radius: 5px;
  box-shadow: 0 0 5px rgba(0,0,0,0.3);
  z-index: 9999;
  font-family: sans-serif;
  font-size: 11px;
}}
.mapLegend .legend-title {{
  font-weight: bold;
  margin-bottom: 5px;
}}
.color-box {{
  display: inline-block;
  width: 12px;
  height: 12px;
  margin-right: 5px;
  vertical-align: middle;
}}
.legend-item {{
  margin-bottom: 3px;
}}
.line-box {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special};
}}
.line-box-2 {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special_2};
}}
.line-box-3 {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special_3};
}}
.line-box-4 {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special_4};
}}
.line-box-5 {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special_5};
}}
.line-box-6 {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special_6};
}}
</style>
"""

mapa.get_root().html.add_child(Element(style))

# Leyenda con las categorías + líneas especiales
legend_html = f"""
<div class='mapLegend'>
  <div class='legend-title'>V. Internet (Mbps)</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[5]};'></span>Mayor que 200 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[4]};'></span>100 a 200 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[3]};'></span>60 a 100 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[2]};'></span>25 a 60 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[1]};'></span>10 a 25 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[0]};'></span>Menor que 10 Mbps</div>
  <hr style="margin:5px 0;">
  <div class='legend-item'><span class='line-box'></span>2019Q4</div>
  <div class='legend-item'><span class='line-box-2'></span>2020Q3</div>
  <div class='legend-item'><span class='line-box-3'></span>2020Q4</div>
  <div class='legend-item'><span class='line-box-4'></span>2021Q1</div>
  <div class='legend-item'><span class='line-box-5'></span>2021Q2</div>
  <div class='legend-item'><span class='line-box-6'></span>2021Q3</div>
</div>
"""

mapa.get_root().html.add_child(Element(legend_html))

folium.LayerControl().add_to(mapa)

mapa.save("mapa_lima_callao_categorias.html")
print("Mapa guardado con líneas para 2019Q4, 2020Q3, 2020Q4, 2021Q1, 2021Q2 y 2021Q3 en la leyenda.")

In [ ]:
import folium
import branca
import geopandas as gpd
from folium import Element
from pyproj import Transformer

# Suponiendo que lima_callao_gdf y filtered_data_2020_q3 están en EPSG:4326

# 1. Reproyectar para calcular el centro
lima_callao_proj = lima_callao_gdf.to_crs(epsg=32718)
centroides = lima_callao_proj.geometry.centroid
centro_y = centroides.y.mean()
centro_x = centroides.x.mean()
transformer = Transformer.from_crs("EPSG:32718", "EPSG:4326", always_xy=True)
centro_lon, centro_lat = transformer.transform(centro_x, centro_y)
centro = [centro_lat, centro_lon]

# Si 'centroid' existe en filtered_data_2020_q3 la removemos
if 'centroid' in filtered_data_2020_q3.columns:
    filtered_data_2020_q3 = filtered_data_2020_q3.drop(columns='centroid')

speeds = filtered_data_2020_q3['avg_d_mbps']

# Definimos los límites de las categorías
colors = [
    '#ffffcc',  # <10
    '#ffeda0',  # 10-25
    '#fed976',  # 25-60
    '#fd8d3c',  # 60-100
    '#f03b20',  # 100-200
    '#800000'   # >200
]

def color_por_categoria(speed):
    if speed > 200:
        return colors[5]
    elif speed > 100:
        return colors[4]
    elif speed > 60:
        return colors[3]
    elif speed > 25:
        return colors[2]
    elif speed > 10:
        return colors[1]
    else:
        return colors[0]

mapa = folium.Map(location=centro, zoom_start=11, tiles="cartodbpositron")

# Añadir las grillas
folium.GeoJson(
    data=filtered_data_2020_q3.to_json(),
    name="Grillas de Velocidad",
    style_function=lambda feature: {
        'fillColor': color_por_categoria(feature['properties']['avg_d_mbps']),
        'color': color_por_categoria(feature['properties']['avg_d_mbps']),
        'weight': 0.5,
        'fillOpacity': 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=['avg_d_mbps', 'PROVINCIA'],
        aliases=['Velocidad (Mbps):', 'Provincia:']
    )
).add_to(mapa)

# Colores para los distintos grupos de distritos
line_color_special = '#67001F'   # SAN MIGUEL y PUEBLO LIBRE
line_color_special_2 = '#980043' # SURQUILLO, MAGDALENA DEL MAR, JESUS MARIA, LINCE, BREÑA
line_color_special_3 = '#6A51A3' # SANTIAGO DE SURCO, MIRAFLORES, etc.
line_color_special_4 = '#045A8D' # LA VICTORIA, SAN LUIS, BARRANCO
line_color_special_5 = '#016C59' # VILLA MARIA DEL TRIUNFO, etc.
line_color_special_6 = '#C51B7D' # COMAS, RIMAC, SANTA ANITA, SAN BORJA, CARABAYLLO
line_color_special_7 = '#5C6BC0' # PUENTE PIEDRA

def estilo_distritos(feature):
    distrito = feature['properties']['DISTRITO']
    if distrito in ["SAN MIGUEL", "PUEBLO LIBRE"]:
        return {
            'fillColor': 'none',
            'color': line_color_special,
            'weight': 2,
            'fillOpacity': 0.0
        }
    elif distrito in ["SURQUILLO", "MAGDALENA DEL MAR", "JESUS MARIA", "LINCE", "BREÑA"]:
        return {
            'fillColor': 'none',
            'color': line_color_special_2,
            'weight': 2,
            'fillOpacity': 0.0
        }
    elif distrito in ["SANTIAGO DE SURCO", "MIRAFLORES", "CHORRILLOS", "LIMA",
                      "SAN ISIDRO", "CALLAO", "LOS OLIVOS", "LA MOLINA",
                      "EL AGUSTINO", "SAN MARTIN DE PORRES", "SAN JUAN DE LURIGANCHO"]:
        return {
            'fillColor': 'none',
            'color': line_color_special_3,
            'weight': 2,
            'fillOpacity': 0.0
        }
    elif distrito in ["LA VICTORIA", "SAN LUIS", "BARRANCO"]:
        return {
            'fillColor': 'none',
            'color': line_color_special_4,
            'weight': 2,
            'fillOpacity': 0.0
        }
    elif distrito in ["VILLA MARIA DEL TRIUNFO", "VILLA EL SALVADOR", "INDEPENDENCIA",
                      "CARMEN DE LA LEGUA REYNOSO", "ATE", "SAN JUAN DE MIRAFLORES"]:
        return {
            'fillColor': 'none',
            'color': line_color_special_5,
            'weight': 2,
            'fillOpacity': 0.0
        }
    elif distrito in ["COMAS", "RIMAC", "SANTA ANITA", "SAN BORJA", "CARABAYLLO"]:
        return {
            'fillColor': 'none',
            'color': line_color_special_6,
            'weight': 2,
            'fillOpacity': 0.0
        }
    elif distrito in ["PUENTE PIEDRA"]:
        return {
            'fillColor': 'none',
            'color': line_color_special_7,
            'weight': 2,
            'fillOpacity': 0.0
        }
    else:
        return {
            'fillColor': 'none',
            'color': '#555555',
            'weight': 1,
            'fillOpacity': 0.0
        }

folium.GeoJson(
    data=lima_callao_gdf.to_json(),
    name="Distritos de Lima y Callao",
    style_function=estilo_distritos,
    tooltip=folium.GeoJsonTooltip(
        fields=['DISTRITO', 'PROVINCIA'],
        aliases=['Distrito:', 'Provincia:']
    )
).add_to(mapa)

# Estilo CSS usando f-string
style = f"""
<style>
.mapLegend {{
  position: absolute;
  top: 350px;
  right: 125px;
  background-color: rgba(255, 255, 255, 0.8);
  padding: 5px;
  border-radius: 5px;
  box-shadow: 0 0 5px rgba(0,0,0,0.3);
  z-index: 9999;
  font-family: sans-serif;
  font-size: 11px;
}}
.mapLegend .legend-title {{
  font-weight: bold;
  margin-bottom: 5px;
}}
.color-box {{
  display: inline-block;
  width: 12px;
  height: 12px;
  margin-right: 5px;
  vertical-align: middle;
}}
.legend-item {{
  margin-bottom: 3px;
}}
.line-box {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special};
}}
.line-box-2 {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special_2};
}}
.line-box-3 {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special_3};
}}
.line-box-4 {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special_4};
}}
.line-box-5 {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special_5};
}}
.line-box-6 {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special_6};
}}
.line-box-7 {{
  display: inline-block;
  vertical-align: middle;
  width: 20px;
  height: 2px;
  margin-right: 5px;
  background: {line_color_special_7};
}}
</style>
"""

mapa.get_root().html.add_child(Element(style))

# Leyenda con las categorías + líneas especiales
legend_html = f"""
<div class='mapLegend'>
  <div class='legend-title'>V. Internet (Mbps)</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[5]};'></span>Mayor que 200 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[4]};'></span>100 a 200 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[3]};'></span>60 a 100 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[2]};'></span>25 a 60 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[1]};'></span>10 a 25 Mbps</div>
  <div class='legend-item'><span class='color-box' style='background:{colors[0]};'></span>Menor que 10 Mbps</div>
  <hr style="margin:5px 0;">
  <div class='legend-item'><span class='line-box'></span>2019Q4</div>
  <div class='legend-item'><span class='line-box-2'></span>2020Q3</div>
  <div class='legend-item'><span class='line-box-3'></span>2020Q4</div>
  <div class='legend-item'><span class='line-box-4'></span>2021Q1</div>
  <div class='legend-item'><span class='line-box-5'></span>2021Q2</div>
  <div class='legend-item'><span class='line-box-6'></span>2021Q3</div>
  <div class='legend-item'><span class='line-box-7'></span>2021Q4</div>
</div>
"""

mapa.get_root().html.add_child(Element(legend_html))

folium.LayerControl().add_to(mapa)

mapa.save("mapa_lima_callao_categorias.html")
print("Mapa guardado con líneas para 2019Q4, 2020Q3, 2020Q4, 2021Q1, 2021Q2, 2021Q3 y 2021Q4 en la leyenda.")


In [ ]:
filtered_data_2020_q3 = quadkeys_final_final1[(quadkeys_final_final1['year'] == 2021) & (quadkeys_final_final1['quarter'] == 4)]

In [ ]:
mapa

In [ ]:

# Especifica el nombre del archivo de salida (GeoJSON)
output_geojson = 'combined_data.geojson'

# Guardar el archivo en formato GeoJSON
combined_data.to_file(output_geojson, driver='GeoJSON')

print(f"Archivo convertido a GeoJSON y guardado como {output_geojson}")